### Data cleaning and analysis

In [1]:
# Importing dependencies
import numpy as np
import pandas as pd
import os

In [2]:
# Reading in the training data set
training_data = pd.read_csv("train.csv")

# Reading in the testing data set
testing_data = pd.read_csv("test.csv")

# Reading in the provided submission's predictions
provided_prediction = pd.read_csv("gender_submission.csv")

combined=training_data.append(testing_data)
combined.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0.0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0.0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0.0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


### Testing the example submission file
##### This file predicted all men died and all women survived

In [3]:
# Testing the provided gender_submission.csv test file
women = training_data.loc[training_data.Sex == 'female']["Survived"]
survival_rate_women = sum(women)/len(women)

print("% of women who survived:", survival_rate_women)

% of women who survived: 0.7420382165605095


In [4]:
# Obviously the provided model is not sufficient for predicting survival.
# Let's check the men's survival rate anyways.
men = training_data.loc[training_data.Sex == 'male']["Survived"]
survival_rate_men = sum(men)/len(men)

print("% of men who survived:", survival_rate_men)

% of men who survived: 0.18890814558058924


In [5]:
# The disparity here is interesting.
# Perhaps something to look into.
# Let us check on how first class tickets affect survival rates
first_class = training_data.loc[training_data.Pclass == 1]["Survived"]
survival_rate_first_class = sum(first_class)/len(first_class)

print("% of first class passengers who survived:", survival_rate_first_class)

% of first class passengers who survived: 0.6296296296296297


In [6]:
# Let us check on how second class tickets affect survival rates
second_class = training_data.loc[training_data.Pclass == 2]["Survived"]
survival_rate_second_class = sum(second_class)/len(second_class)

print("% of second class passengers who survived:", survival_rate_second_class)

% of second class passengers who survived: 0.47282608695652173


In [7]:
# Let us check on how third class tickets affect survival rates
third_class = training_data.loc[training_data.Pclass == 3]["Survived"]
survival_rate_third_class = sum(third_class)/len(third_class)

print("% of third class passengers who survived:", survival_rate_third_class)

% of third class passengers who survived: 0.24236252545824846


In [8]:
# This is also important to take note of and will certainly be an important factor in our machine learning model.
# Next we will take a look at the effect of embarking from Cherbourg on the survival rate
cherbourg = training_data.loc[training_data.Embarked == "C"]["Survived"]
survival_rate_cherbourg = sum(cherbourg)/len(cherbourg)

print("% of passengers who embarked at Cherbourg and survived:", survival_rate_cherbourg)

% of passengers who embarked at Cherbourg and survived: 0.5535714285714286


In [9]:
# Now we will take a look at the effect of embarking from Queenstown on the survival rate
queenstown = training_data.loc[training_data.Embarked == "Q"]["Survived"]
survival_rate_queenstown = sum(queenstown)/len(queenstown)

print("% of passengers who embarked at Queenstown and survived:", survival_rate_queenstown)

% of passengers who embarked at Queenstown and survived: 0.38961038961038963


In [10]:
# Now we will take a look at the effect of embarking from Southampton on the survival rate
southampton = training_data.loc[training_data.Embarked == "S"]["Survived"]
survival_rate_southampton = sum(southampton)/len(southampton)

print("% of passengers who embarked at Southampton and survived:", survival_rate_southampton)

% of passengers who embarked at Southampton and survived: 0.33900928792569657


### Cleaning up empty values

In [11]:
# Testing for NaN in columns. I added data for Martha Evelyn Stone's 
# and Emilia Icard's port of embarkation manually by finding their information elsewhere
print(combined.isna().sum())
print('Missing values Percentage: \n\n', round (combined.isnull().sum().sort_values(ascending=False)/len(combined)*100,1))

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          0
dtype: int64
Missing values Percentage: 

 Cabin          77.5
Survived       31.9
Age            20.1
Fare            0.1
PassengerId     0.0
Pclass          0.0
Name            0.0
Sex             0.0
SibSp           0.0
Parch           0.0
Ticket          0.0
Embarked        0.0
dtype: float64


In [12]:
# Due to the extreme missing values in the Cabin column. It does not make sense to fill or keep the column. For now I am going to drop it.
training_data = training_data.drop(columns = "Cabin")
testing_data = testing_data.drop(columns = "Cabin")

In [13]:
# Due to the quantity of missing age values, it does not seem prudent to search outside of 
# this dataset for the missing ages. Many would likely be found, but the exact age does not 
# particularly matter. Getting an idea of the age is more important, i.e. whether they are 
# an infant/child/adult/senior

In [14]:
# Searching for male children who do not have an age in the training data
display(combined[(combined.Age.isnull()) & (combined.Name.str.contains('Master'))])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
65,66,1.0,3,"Moubarek, Master. Gerios",male,NaN,1,1,2661,15.2458,NaN,C
159,160,0.0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,CA. 2343,69.5500,NaN,S
176,177,0.0,3,"Lefebre, Master. Henry Forbes",male,NaN,3,1,4133,25.4667,NaN,S
709,710,1.0,3,"Moubarek, Master. Halim Gonios (""William George"")",male,NaN,1,1,2661,15.2458,NaN,C
244,1136,NaN,3,"Johnston, Master. William Arthur Willie""""",male,NaN,1,2,W./C. 6607,23.4500,NaN,S
339,1231,NaN,3,"Betros, Master. Seman",male,NaN,0,0,2622,7.2292,NaN,C
344,1236,NaN,3,"van Billiard, Master. James William",male,NaN,1,1,A/5. 851,14.5000,NaN,S
417,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [15]:
# Finding the mean age of these children
print(training_data[training_data.Name.str.contains('Master')]['Age'].mean())

4.574166666666667


In [16]:
# Determining whether the children are travelling alone by checking parch == 0
display((combined[(combined.Age.isnull()) & (combined.Name.str.contains('Master')) & (combined.Parch==0)]))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
339,1231,NaN,3,"Betros, Master. Seman",male,NaN,0,0,2622,7.2292,NaN,C


In [17]:
# Fixing the one instance
testing_data.loc[testing_data.PassengerId==1231,'Age']=14

In [18]:
# The previous analysis can most likely apply to multiple groups,
# so I will pull out all of the titles to help impute missing ages.
training_data['Title'], testing_data['Title'] = [df.Name.str.extract \
        (' ([A-Za-z]+)\.', expand=False) for df in [training_data, testing_data]]

In [19]:
# Using groupby to filter across multiple columns we can see the average age in this context
training_data.groupby(['Title', 'Pclass'])['Age'].agg(['mean', 'count'])

mean  count
Title    Pclass                  
Capt     1       70.000000      1
Col      1       58.000000      2
Countess 1       33.000000      1
Don      1       40.000000      1
Dr       1       43.750000      4
         2       38.500000      2
Jonkheer 1       38.000000      1
Lady     1       48.000000      1
Major    1       48.500000      2
Master   1        5.306667      3
         2        2.258889      9
         3        5.350833     24
Miss     1       30.000000     45
         2       22.390625     32
         3       16.123188     69
Mlle     1       24.000000      2
Mme      1       24.000000      1
Mr       1       41.580460     87
         2       32.768293     82
         3       28.724891    229
Mrs      1       40.882353     34
         2       33.682927     41
         3       33.515152     33
Ms       2       28.000000      1
Rev      2       43.166667      6
Sir      1       49.000000      1

In [20]:
# Dictionary that maps the titles to a more cocise number
TitleDict = {"Capt": "Officer","Col": "Officer","Major": "Officer","Jonkheer": "Royalty", \
             "Don": "Royalty", "Sir" : "Royalty","Dr": "Royalty","Rev": "Royalty", \
             "Countess":"Royalty", "Mme": "Mrs", "Mlle": "Miss", "Ms": "Mrs","Mr" : "Mr", \
             "Mrs" : "Mrs","Miss" : "Miss","Master" : "Master","Lady" : "Royalty"}

In [21]:
# Mapping the dictionary to the training data set
training_data['Title'], testing_data['Title'] = [df.Title.map(TitleDict) for df in [training_data, testing_data]]

# Now reprint the groups
training_data.groupby(['Title', 'Pclass'])['Age'].agg(['mean', 'count'])

mean  count
Title   Pclass                  
Master  1        5.306667      3
        2        2.258889      9
        3        5.350833     24
Miss    1       29.744681     47
        2       22.390625     32
        3       16.123188     69
Mr      1       41.580460     87
        2       32.768293     82
        3       28.724891    229
Mrs     1       40.400000     35
        2       33.547619     42
        3       33.515152     33
Officer 1       56.600000      5
Royalty 1       42.555556      9
        2       42.000000      8

In [22]:
# Checking for children with no parents
combined=training_data.append(testing_data)
display(training_data[training_data.Title.isnull()])
display(testing_data[testing_data.Title.isnull()])

# There is Dona which is royalty which is not covered in test_data
testing_data.at[414,'Title'] = 'Royalty'
display(testing_data[testing_data.Title.isnull()])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9,C,NaN


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title


In [23]:
# Testing how to deal with missing values for young women
print ("Avg age of 'Miss' Title", round(training_data[training_data.Title=="Miss"]['Age'].mean()))

print ("Avg age of 'Miss' Title travelling without Parents", round(training_data[(training_data.Title=="Miss") & (training_data.Parch==0)]['Age'].mean()))

print ("Avg age of 'Miss' Title travelling with Parents", round(training_data[(training_data.Title=="Miss") & (training_data.Parch!=0)]['Age'].mean()), '\n')

Avg age of 'Miss' Title 22
Avg age of 'Miss' Title travelling without Parents 28
Avg age of 'Miss' Title travelling with Parents 12 



In [24]:
# Determining the number of passengers per ticket in the training data set
display(combined[combined.Fare.isnull()])
for df in [training_data, testing_data, combined]:
    df['PeoplePerTicket']=df['Ticket'].map(combined['Ticket'].value_counts())
    df['FarePerPerson']=df['Fare']/df['PeoplePerTicket']

print('Mean fare for this category: ', training_data[(training_data.Embarked=='S') & (training_data.Pclass==3)]['FarePerPerson'].mean())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
152,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,S,Mr


Mean fare for this category:  7.382647773383372


In [25]:
# This version is more accurate
testing_data.loc[testing_data.Fare.isnull(), ['Fare','FarePerPerson']] = round(training_data[(training_data.Embarked=='S') & (training_data.Pclass==3) & (training_data.PeoplePerTicket==1)]['Fare'].mean(),1)

In [26]:
# Checking for missing embarked again
display(combined[combined.Embarked.isnull()])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,PeoplePerTicket,FarePerPerson


In [27]:
# Mean Median and count of ages by title, gender, and class
print(training_data.groupby(['Pclass','Sex','Title'])['Age'].agg({'mean', 'median', 'count'}))

# New title: FemaleChild created for the numerous females with parents and Nan age
for df in [training_data, testing_data, combined]:
    df.loc[(df.Title=='Miss') & (df.Parch!=0) & (df.PeoplePerTicket>1), 'Title']="FemaleChild"

display(combined[(combined.Age.isnull()) & (combined.Title=='FemaleChild')])

                            mean  median  count
Pclass Sex    Title                            
1      female Miss     29.744681    30.0     47
              Mrs      40.400000    40.0     35
              Royalty  43.333333    48.0      3
       male   Master    5.306667     4.0      3
              Mr       41.580460    40.0     87
              Officer  56.600000    56.0      5
              Royalty  42.166667    42.0      6
2      female Miss     22.390625    24.0     32
              Mrs      33.547619    31.5     42
       male   Master    2.258889     1.0      9
              Mr       32.768293    31.0     82
              Royalty  42.000000    46.5      8
3      female Miss     16.123188    18.0     69
              Mrs      33.515152    31.0     33
       male   Master    5.350833     4.0     24
              Mr       28.724891    26.0    229


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,PeoplePerTicket,FarePerPerson
128,129,1.0,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,C,FemaleChild,3,7.452767
180,181,0.0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.5500,S,FemaleChild,11,6.322727
229,230,0.0,3,"Lefebre, Miss. Mathilde",female,NaN,3,1,4133,25.4667,S,FemaleChild,5,5.093340
409,410,0.0,3,"Lefebre, Miss. Ida",female,NaN,3,1,4133,25.4667,S,FemaleChild,5,5.093340
485,486,0.0,3,"Lefebre, Miss. Jeannie",female,NaN,3,1,4133,25.4667,S,FemaleChild,5,5.093340
792,793,0.0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,CA. 2343,69.5500,S,FemaleChild,11,6.322727
863,864,0.0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,S,FemaleChild,11,6.322727
888,889,0.0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S,FemaleChild,4,5.862500
188,1080,NaN,3,"Sage, Miss. Ada",female,NaN,8,2,CA. 2343,69.5500,S,FemaleChild,11,6.322727


In [28]:
# Creating a lookup table
grp = training_data.groupby(['Pclass','Sex','Title'])['Age'].mean().reset_index()[['Sex', 'Pclass', 'Title', 'Age']]

print('\n', 'We converted the series object to: ', type(grp))


 We converted the series object to:  <class 'pandas.core.frame.DataFrame'>


In [29]:
# Testing the lookup table
print('\n', 'Mean for desired parameters: ', grp[(grp.Pclass==2) & (grp.Sex=='male') & (grp.Title=='Master')]['Age'].values[0])


 Mean for desired parameters:  2.2588888888888885


In [30]:
# Now we create a function to fill in the missing ages in the training data set
def fill_age(x):
    return grp[(grp.Pclass==x.Pclass)&(grp.Sex==x.Sex)&(grp.Title==x.Title)]['Age'].values[0]

In [31]:
# Now we can call the function
training_data['Age'], testing_data['Age'] = [df.apply(lambda x: fill_age(x) if np.isnan(x['Age']) else x['Age'], axis=1) for df in [training_data, testing_data]]

# Combine one last time 
combined=training_data.append(testing_data)

In [32]:
# Checking for remaining missing data. All that should remain is the undetermined survival status of the testing data set
combined.isna().sum()

PassengerId          0
Survived           418
Pclass               0
Name                 0
Sex                  0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 0
Embarked             0
Title                0
PeoplePerTicket      0
FarePerPerson        0
dtype: int64